**Función:** Entrenamiento de la CNN línea base con dataset 10k con 1 canal (gris).

**Requiere:**
- ../../datasets/X_train_DATASET10KGRIS_x_float.pkl
- ../../datasets/X_test_DATASET10KGRIS_x_float.pkl
- ../../datasets/y_train_DATASET10KGRIS.pkl
- ../../datasets/y_test_DATASET10KGRIS.pkl

**Crea:** Fichero con el modelo entrenado (epoch con menor val_loss).
- modelos/tf_model_\<run_id\>

In [ ]:
ejecutadoEnGoogleDrive = False # True si se ejecuta en Google Drive

In [ ]:
if ejecutadoEnGoogleDrive:
    from google.colab import drive
    drive.mount('/content/drive')
    PATH = '/content/drive/clasificacion_imagenes/'
    ruta_datasets = '/content/drive/clasificacion_imagenes/datasets/'
else:
    PATH = '../../1_Setup_MLOps'
    ruta_datasets = '../../datasets/'

%cd {PATH}

In [2]:
# !pip install mlflow
import mlflow
#from google.colab import drive
import numpy as np
from numpy.random import seed
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from keras import backend as K
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Convolution2D, Conv2D, MaxPooling2D
from tensorflow import keras
from tensorflow.random import set_seed
import time

2022-09-17 18:02:24.556091: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-17 18:02:24.556232: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
# Fijar random state para obtener resultados reproductibles
seed(42) # So long and thanks for all the fish
set_seed(42)

In [4]:
!pwd

/drive/Experimento_3


In [5]:
# ir a la carpeta raiz del proyecto (donde se encuentra mlruns)
%cd ..

/drive


In [1]:
# Definir quién está ejecutando (para no machacar el MLflow de los compañeros)
# Descomentar la linea que aplique
# yo = 'Pedro_T'
# yo = 'Toni_V'
# yo = 'Carlos_H'
# yo = 'Carlos_C'
print('Notebook ejecutado por {}'.format(yo))

Notebook ejecutado por Carlos_H


In [7]:
# Este run pertenence al experimento 2, que tiene el id "1" (string)
experiment_id = "2"
mlflow.set_experiment(experiment_id = experiment_id) 

<Experiment: artifact_location='mlruns', experiment_id='2', lifecycle_stage='active', name='Experimento 3 - evaluar impacto de los canales', tags={'Dataset': 'DATASET 10K 224 GRIS',
 'Experimento': '3',
 'Modelo': 'CNN basica (baseline)'}>

In [8]:
# OPCIÓN 1: crear run
Este texto garantiza que no creamos accidentalmente un nuevo run
run = mlflow.start_run(
  experiment_id=experiment_id,
  tags={"Autor": yo}
)
print('Vamos empezar el run {} en el experimento {}. Este run está {}.'.format(run.info.run_id, experiment_id, run.info.status))

Vamos empezar el run 43e9f8094e20474d8d136c1276ec82a6 en el experimento 2. Este run está RUNNING.


In [7]:
# OPCIÓN 2: reanudar run
run_id = '43e9f8094e20474d8d136c1276ec82a6' # por ej. "69bfdf296f384a75ada2ae55f9dc7672" (ver en la carpeta mlruns/0)
run = mlflow.start_run(run_id=run_id)
print('Vamos empezar el run {} en el experimento id_{}. Este run está {}.'.format(run.info.run_id, experiment_id, run.info.status))

Vamos empezar el run 6b6eff59143148e78dfe1a10efc7f71e en el experimento id_1. Este run está RUNNING.


In [9]:
# activar auto logging
mlflow.tensorflow.autolog()

Formato esperado:
- X_train: (N, 224, 224, 3) float32 (entre 0. y 1.)
- y_train: (N, 5) float32 (0. o 1.)
- X_test: (P, 224, 224, 3) float32 (entre 0. y 1.)
- y_test: (P, 5) float32 (0. o 1.)

In [10]:
!ls datasets/DATASET10KGRIS

X_test.pkl  X_train.pkl  y_test.pkl  y_train.pkl


In [12]:
X_train = pickle.load(open(PATH + 'X_train_DATASET10KGRIS.pkl'.format(ruta_datasets),'rb'))
X_test = pickle.load(open(PATH + 'X_test_DATASET10KGRIS.pkl'.format(ruta_datasets),'rb'))
y_train = pickle.load(open(PATH + 'y_train_DATASET10KGRIS.pkl'.format(ruta_datasets),'rb'))
y_test = pickle.load(open(PATH + 'y_test_DATASET10KGRIS.pkl'.format(ruta_datasets),'rb'))

In [13]:
categories = ['food', 'inside', 'outside', 'drink', 'menu']

In [14]:
print("X_train: {} {}".format(X_train.shape, X_train.dtype))
print("y_train: {} {}".format(y_train.shape, y_train.dtype))
print("X_test: {} {}".format(X_test.shape, X_test.dtype))
print("y_test: {} {}".format(y_test.shape, y_test.dtype))

X_train: (6650, 224, 224) float32
y_train: (6650, 5) float32
X_test: (1662, 224, 224) float32
y_test: (1662, 5) float32


In [15]:
# Dimensiones de la imagen de entrada
img_rows, img_cols, img_ch = 224, 224, 3
num_labels = 5

In [16]:
# test de los datos de entrada
if   (X_train.shape[1] != img_rows) \
  or (X_train.shape[2] != img_cols)\
  or (y_train.shape[1] != num_labels)\
  or (X_train.dtype != 'float32')\
  or (X_test.shape[1] != img_rows) \
  or (X_test.shape[2] != img_cols)\
  or (y_test.shape[1] != num_labels)\
  or (X_test.dtype != 'float32')\
  or (y_train.dtype != 'float32')\
  or (y_test.dtype != 'float32')\
  or (X_train.shape[0] != y_train.shape[0])\
  or (X_test.shape[0] != y_test.shape[0]\
  or type(X_train) != np.ndarray\
  or type(X_test) != np.ndarray\
  or type(y_train) != np.ndarray\
  or type(y_test) != np.ndarray):
  raise Exception("El dataset no tiene las dimensiones esperadas!")
else:
  print('El dataset tiene las dimensiones esperadas')

El dataset tiene las dimensiones esperadas


In [17]:
IMG_SIZE = 224
epochs = 40
batch_size=64
validation_split = 0.2

In [25]:
model = Sequential()
model.add(Conv2D(10, (12, 12), padding='same', input_shape=(IMG_SIZE,IMG_SIZE,1)))
model.add(Activation('relu'))
model.add(Conv2D(IMG_SIZE, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(4, 4)))
model.add(Dropout(0.25))

model.add(Conv2D(IMG_SIZE, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(IMG_SIZE, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))
opt = keras.optimizers.Nadam(learning_rate=0.0001, schedule_decay=1e-6)

In [26]:
# las métricas han sido quitadas de Keras en 2.0
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [27]:
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['acc', f1_m, precision_m, recall_m])

In [28]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 224, 224, 10)      1450      
                                                                 
 activation_6 (Activation)   (None, 224, 224, 10)      0         
                                                                 
 conv2d_6 (Conv2D)           (None, 222, 222, 224)     20384     
                                                                 
 activation_7 (Activation)   (None, 222, 222, 224)     0         
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 55, 55, 224)      0         
 2D)                                                             
                                                                 
 dropout_3 (Dropout)         (None, 55, 55, 224)       0         
                                                      

In [29]:
callback = keras.callbacks.EarlyStopping(monitor='val_loss',
                                         patience=8,
                                         restore_best_weights=True)

In [30]:
start_time = time.time()
history = model.fit(X_train, y_train,
                    epochs = epochs,
                    batch_size=batch_size,
                    validation_data=(X_test, y_test),
                    callbacks=[callback],
                    verbose=1)
end_time = time.time()

Epoch 1/40


2022-09-17 18:05:02.474179: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2826141696 exceeds 10% of free system memory.
2022-09-17 18:05:41.076590: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2826141696 exceeds 10% of free system memory.


  1/104 [..............................] - ETA: 1:38:46 - loss: 1.6313 - acc: 0.0625 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00

2022-09-17 18:05:56.931555: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2826141696 exceeds 10% of free system memory.
2022-09-17 18:06:05.933243: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2826141696 exceeds 10% of free system memory.


  2/104 [..............................] - ETA: 29:20 - loss: 1.4988 - acc: 0.2891 - f1_m: 0.2578 - precision_m: 0.2578 - recall_m: 0.2578              

2022-09-17 18:06:13.439633: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 2826141696 exceeds 10% of free system memory.


104/104 [==============================] - 1680s 16s/step - loss: 1.0438 - acc: 0.6217 - f1_m: 0.5654 - precision_m: 0.6823 - recall_m: 0.4908 - val_loss: 1.1115 - val_acc: 0.5969 - val_f1_m: 0.4419 - val_precision_m: 0.6491 - val_recall_m: 0.3438
Epoch 2/40
104/104 [==============================] - 1518s 15s/step - loss: 0.9461 - acc: 0.6720 - f1_m: 0.6360 - precision_m: 0.7289 - recall_m: 0.5672 - val_loss: 0.9360 - val_acc: 0.6865 - val_f1_m: 0.6325 - val_precision_m: 0.7411 - val_recall_m: 0.5523
Epoch 3/40
104/104 [==============================] - 1522s 15s/step - loss: 0.8692 - acc: 0.6887 - f1_m: 0.6670 - precision_m: 0.7503 - recall_m: 0.6035 - val_loss: 0.7663 - val_acc: 0.7232 - val_f1_m: 0.6961 - val_precision_m: 0.7380 - val_recall_m: 0.6641
Epoch 4/40
104/104 [==============================] - 1511s 15s/step - loss: 0.7997 - acc: 0.7153 - f1_m: 0.6961 - precision_m: 0.7757 - recall_m: 0.6353 - val_loss: 0.6855 - val_acc: 0.7527 - val_f1_m: 0.7176 - val_precision_m: 0.785

INFO:tensorflow:Assets written to: /tmp/tmpfergpo19/model/data/model/assets


INFO:tensorflow:Assets written to: /tmp/tmpfergpo19/model/data/model/assets
2022/09/18 01:05:56 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpfergpo19/model, flavor: keras), fall back to return ['tensorflow==2.9.1', 'keras==2.9.0']. Set logging level to DEBUG to see the full traceback.


In [35]:
model.save(model_path + 'tf_model_exp1_{}'.format(run.info.run_id))

INFO:tensorflow:Assets written to: Experimento_3/modelos/tf_model_exp1_43e9f8094e20474d8d136c1276ec82a6/assets


INFO:tensorflow:Assets written to: Experimento_3/modelos/tf_model_exp1_43e9f8094e20474d8d136c1276ec82a6/assets


In [ ]:
mlflow.end_run()